# Darcy equation

In this tutorial we present how to solve a Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = -{ \rho g \nabla y}\\
\nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \qquad p = \rho g \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [2]:
import shutil
import os

import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

In [3]:
N = 10

rho = 1000
g = pp.GRAVITY_ACCELERATION

In [4]:
output_directory = 'output_gravity'

We create now the grid, since we will use a Raviart-Thomas approximation for ${q}$ we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 1d and 3d. PyGeoN works with mixed-dimensional grids, so we need to convert the grid.

In [5]:
subdomain = pp.StructuredTriangleGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([subdomain])

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [6]:
key = "flow"

bc_value = []
bc_essential = []

gravity = []

RT0 = pg.RT0(key)
P0 = pg.PwConstants(key)

def g_func(x): return np.array([0, -rho * g, -rho * g])

for subdomain, data in mdg.subdomains(return_data=True):
    g_proj = RT0.interpolate(subdomain, g_func)
    gravity.append(RT0.assemble_mass_matrix(subdomain) @ g_proj)

    # permeability tensor
    perm = pp.SecondOrderTensor(np.ones(subdomain.num_cells))
    parameters = {
        "second_order_tensor": perm,
    }
    pp.initialize_data(subdomain, data, key, parameters)
        
    # with the following steps we identify the portions of the boundary
    # to impose the boundary conditions
    left_right = np.logical_or(subdomain.face_centers[0, :] == 0,  subdomain.face_centers[0, :] == 1)
    
    bottom = subdomain.face_centers[1, :] == 0
    top    = subdomain.face_centers[1, :] == 1
    
    def p_bc(x): return rho * g

    bc_value.append(- RT0.assemble_nat_bc(subdomain, p_bc, bottom))

    essential_pressure_dofs = np.zeros(P0.ndof(subdomain), dtype=bool)
    bc_essential.append(np.hstack((left_right, essential_pressure_dofs)))

Once the data are assigned to the mixed-dimensional grid, we construct the matrices. In particular, the linear system associated with the equation is given as
$$
\left(
\begin{array}{cc} 
M & B^\top\\
B & 0
\end{array}
\right)
\left(
\begin{array}{c} 
q\\ 
p
\end{array}
\right)
=\left(
\begin{array}{c} 
p_{\partial} + M_p g\\ 
0
\end{array}
\right)
$$<br>
where $p_{\partial}$ is the vector associated to the pressure boundary contions. To construct the saddle-point problem, we rely on the `scipy.sparse` function `bmat`. Once the matrix is created, we also construct the right-hand side containing the boundary conditions.

In [7]:
# construct the local matrices
mass = pg.face_mass(mdg)
div = pg.cell_mass(mdg) @ pg.div(mdg)

# assemble the saddle point problem
spp = sps.bmat([[mass, -div.T], 
                [ div,  None]], format="csc")

# get the degrees of freedom for each variable
dof_p, dof_q = div.shape

# assemble the right-hand side
rhs = np.zeros(dof_p + dof_q)

rhs[:dof_q] += np.hstack(bc_value)
rhs[:dof_q] += np.hstack(gravity)

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (flux boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom. Once the problem is solved, we extract the two solutions $q$ and $p$.

In [7]:
# solve the problem
ls = pg.LinearSystem(spp, rhs)
ls.flag_ess_bc(np.hstack(bc_essential), np.zeros(dof_q + dof_p))
x = ls.solve()

Since the computed $q$ is one value per facet of the grid, for visualization purposes we project the flux in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [8]:
def export(current_sol) -> None:
    save = pp.Exporter(mdg, "sol", folder_name="output_gravity")
    proj_q = RT0.eval_at_cell_centers(subdomain)
    proj_p =  P0.eval_at_cell_centers(subdomain)

    q = current_sol[:dof_q]
    p = current_sol[-dof_p:]

    for _, data in mdg.subdomains(return_data=True):
        data[pp.STATE] = {"cell_q": (proj_q @ q).reshape((3, -1), order="F"), 
                          "cell_p": (proj_p @ p)}
    
    save.write_vtu(["cell_q", "cell_p"])

    return

In [9]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [10]:
# post process variables
export(x)